In [10]:
# Luca Soltero ECON 499

In [2]:
import pandas as pd
import sqlite3

repo_dir = "your path here"
conn = sqlite3.connect(repo_dir)
curs = conn.cursor()
pd.set_option('display.max_columns', None)

qry = """WITH prelim AS (
    SELECT
        dir.dealid,
        dir.investorfundid,
        dir.InvestorInvestmentAmount,
        dir.NumberOfSharesAcquired AS NumberOfSharesAcquired_DIR,
        d.NumberOfSharesAcquired AS NumberOfSharesAcquired_D,
        d.PercentAcquired,
        d.DealSize,
        d.SeriesOfStock,
        d.DealDate,
        d.CompanyID,
        d.DealID
    FROM
        DealInvestorRelation dir
    LEFT JOIN
        Deal d ON dir.dealid = d.DealID
),
joined AS (
    SELECT
        p.dealid,
        p.investorfundid,
        p.InvestorInvestmentAmount,
        p.NumberOfSharesAcquired_DIR,
        p.NumberOfSharesAcquired_D,
        p.PercentAcquired,
        p.DealSize,
        p.DealDate,
        p.CompanyID,
        p.DealID,
        f.FundCountry,
        CASE
            WHEN p.SeriesOfStock LIKE '%Seed%' THEN 'Seed'
            WHEN p.SeriesOfStock LIKE '%A%' THEN 'A'
            WHEN p.SeriesOfStock LIKE '%B%' THEN 'B'
            WHEN p.SeriesOfStock LIKE '%C%' THEN 'C'
            WHEN p.SeriesOfStock LIKE '%D%' THEN 'D'
            WHEN p.SeriesOfStock LIKE '%E%' THEN 'E'
            WHEN p.SeriesOfStock LIKE '%F%' THEN 'F'
            WHEN p.SeriesOfStock LIKE '%G%' THEN 'G'
            ELSE 'Null'
        END AS SeedType
    FROM
        prelim p
    LEFT JOIN
        Fund f ON p.investorfundid = f.fundid
),
seed_investment_info AS (
    SELECT
        *,
        CASE
            WHEN InvestorInvestmentAmount IS NOT NULL THEN 1
            ELSE 0
        END AS HasInvestedAmount
    FROM
        joined
),
ranked_investments AS (
    SELECT
        *,
        ROW_NUMBER() OVER (PARTITION BY investorfundid, CompanyID ORDER BY DealDate) AS Rank
    FROM
        seed_investment_info
)
SELECT
    *
FROM
    ranked_investments
WHERE
    InvestorInvestmentAmount IS NOT NULL
    AND FundCountry = 'United States'"""

# query sample of interest where fund is in the US and we observe ticket sizes.

df = pd.read_sql_query(qry, conn)
df['DealDate'] = pd.to_datetime(df['DealDate'])
df.rename(columns={'NumberOfSharesAcquired': 'NumberOfSharesAcquired_DIR'}, inplace=True)

27251
23656


In [5]:
# create respective columns
# FundPA refers to the percent acquired by an individual fund in a given deal
# PredictedFundPA_0 refers to the predicted percent acquired by a fund if we do not know the exact amount of shares they bought in a given deal
# FundPA-PredictedFundPA_0 is the difference between the two
df["TotalShares"] = (df["NumberOfSharesAcquired_D"]/df["PercentAcquired"]) * 100
df["FundPA"] = (df["NumberOfSharesAcquired_DIR"]/df["TotalShares"]) * 100
df["PredictedFundPA_0"] = (df["InvestorInvestmentAmount"]/df["DealSize"]) * df["PercentAcquired"]
df["FundPA-PredictedFundPA_0"] = (df["FundPA"] - df["PredictedFundPA_0"]).abs()
rrwApproachScope = df.dropna(subset="PredictedFundPA_0")
print(len(rrwApproachScope["CompanyID"].unique()))

10189


In [13]:
percentiles = [95, 98, 99]
values_at_percentiles = df["FundPA-PredictedFundPA_0"].quantile([p / 100 for p in percentiles])

# Now, print out the values at each percentile
for percentile, value in zip(percentiles, values_at_percentiles):
    print(f"{percentile}% percentile: {value}")

95% percentile: 3.2358231658674548
98% percentile: 7.104407588274123
99% percentile: 9.895429706339462


In [14]:
# Test to see if InvestorOwnership/DealSize * PercentAcquired is a valid way of getting percent ownership
df_0 = df.dropna()
df_0 = df_0[df_0["FundPA-PredictedFundPA_0"] < 3.3]
# Assuming you have actual values in 'FundOwnership' and predicted values in 'PredictedFundOwnership' columns
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
actual_values = df_0['FundPA']
predicted_values = df_0['PredictedFundPA_0']
print("Distribution of Error")
print(df["FundPA-PredictedFundPA_0"].describe())

mae = mean_absolute_error(actual_values, predicted_values)
mse = mean_squared_error(actual_values, predicted_values)
rmse = np.sqrt(mse)
r2 = r2_score(actual_values, predicted_values)
# Print summary
print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2):", r2)

Distribution of Error
count    1.559000e+03
mean     7.541846e-01
std      6.324125e+00
min      0.000000e+00
25%      3.787900e-08
50%      7.129171e-07
75%      4.313681e-02
max      2.091102e+02
Name: FundPA-PredictedFundPA_0, dtype: float64
Mean Absolute Error (MAE): 0.18810037836959576
Mean Squared Error (MSE): 0.3267418411851307
Root Mean Squared Error (RMSE): 0.5716133668705892
R-squared (R2): 0.9951487077622324


In [15]:

# NOW Assume we do not know InvestorInvestedAmount we use the average ticket of each fund
# We will start with Non - First Investment Tickets
NonFirst_ticket = df.groupby(['investorfundid', 'CompanyID']).apply(lambda x: (x.loc[x['Rank'] != 1]['InvestorInvestmentAmount']))
NFT_AVG = NonFirst_ticket.groupby('investorfundid').mean()
NFT_df = df[df["Rank"] != 1]

NFT_df = NFT_df.merge(NFT_AVG, left_on='investorfundid', right_index=True, suffixes=('', '_avg'))
# Rename the column to 'average ticket'
NFT_df.rename(columns={'InvestorInvestedAmount': 'average_ticket'}, inplace=True)
# Merging FT_AVG_df with the original dataframe based on InvestorFundID
#df = df.merge(FT_AVG_df, how='left', on='InvestorFundID')
columns_to_check = ['PercentAcquired', 'DealSize']
NFT_df = NFT_df.dropna(subset=columns_to_check)

# divide the average for that fund by the deal size and multiply by percent acquired
# we also get the abs value of the error
NFT_df["PredictedFundPA_2"] = (NFT_df["InvestorInvestmentAmount_avg"]/NFT_df["DealSize"]) * NFT_df["PercentAcquired"]
NFT_df["FundPA-PredictedFundPA_2"] = (NFT_df["FundPA"] - NFT_df["PredictedFundPA_2"]).abs()

In [16]:
percentiles = [95, 98, 99]
values_at_percentiles = NFT_df["FundPA-PredictedFundPA_2"].quantile([p / 100 for p in percentiles])

# Now, print out the values at each percentile
for percentile, value in zip(percentiles, values_at_percentiles):
    print(f"{percentile}% percentile: {value}")

95% percentile: 16.82545940761664
98% percentile: 45.588342551846
99% percentile: 129.45106676866934


In [17]:
# drop values beyond 95%
NFT_df = NFT_df[NFT_df["FundPA-PredictedFundPA_2"] < 17]
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
# Assuming you have actual values in 'FundOwnership' and predicted values in 'PredictedFundOwnership' columns
actual_values = NFT_df['FundPA']
predicted_values = NFT_df['PredictedFundPA_2']
print("Distribution of Error")
print(NFT_df["FundPA-PredictedFundPA_2"].describe())
print()
mae = mean_absolute_error(actual_values, predicted_values)
mse = mean_squared_error(actual_values, predicted_values)
rmse = np.sqrt(mse)
r2 = r2_score(actual_values, predicted_values)
# Print summary
print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2):", r2)

Distribution of Error
count    416.000000
mean       1.887981
std        3.183524
min        0.000000
25%        0.003231
50%        0.517389
75%        2.301189
max       16.919056
Name: FundPA-PredictedFundPA_2, dtype: float64

Mean Absolute Error (MAE): 1.8879809206216176
Mean Squared Error (MSE): 13.674932342251095
Root Mean Squared Error (RMSE): 3.697963269456728
R-squared (R2): 0.629136957290322


In [18]:
# Now look at approach using average FundInvestment for First Investments exclusively

first_ticket = df.groupby(['investorfundid', 'CompanyID']).apply(lambda x: (x.loc[x['Rank'] == 1]['InvestorInvestmentAmount']))
FT_AVG = first_ticket.groupby('investorfundid').mean()

FT_df = df[df["Rank"] == 1]

FT_df = FT_df.merge(FT_AVG, left_on='investorfundid', right_index=True, suffixes=('', '_avg'))
# Rename the column to 'average ticket'
FT_df.rename(columns={'InvestorInvestedAmount': 'average_ticket'}, inplace=True)
columns_to_check = ['PercentAcquired', 'DealSize']
FT_df = FT_df.dropna(subset=columns_to_check)

FT_df["PredictedFundPA_1"] = (FT_df["InvestorInvestmentAmount_avg"]/FT_df["DealSize"]) * FT_df["PercentAcquired"]
FT_df["FundPA-PredictedFundPA_1"] = (FT_df["FundPA"] - FT_df["PredictedFundPA_1"]).abs()

In [19]:
percentiles = [95, 98, 99]
values_at_percentiles = FT_df["FundPA-PredictedFundPA_1"].quantile([p / 100 for p in percentiles])

# Now, print out the values at each percentile
for percentile, value in zip(percentiles, values_at_percentiles):
    print(f"{percentile}% percentile: {value}")

95% percentile: 22.912842318082706
98% percentile: 55.44967837966915
99% percentile: 105.95806600023046


In [20]:
FT_df = FT_df[FT_df["FundPA-PredictedFundPA_1"] <= 23]
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
# Assuming you have actual values in 'FundOwnership' and predicted values in 'PredictedFundOwnership' columns
actual_values = FT_df['FundPA']
predicted_values = FT_df['PredictedFundPA_1']
print("Distribution of Error")
print(FT_df["FundPA-PredictedFundPA_1"].describe())
print()
# Calculate metrics
mae = mean_absolute_error(actual_values, predicted_values)
mse = mean_squared_error(actual_values, predicted_values)
rmse = np.sqrt(mse)
r2 = r2_score(actual_values, predicted_values)
# Print summary
print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2):", r2)

Distribution of Error
count    1066.000000
mean        2.922377
std         4.339760
min         0.000000
25%         0.085205
50%         1.022308
75%         4.020931
max        22.925306
Name: FundPA-PredictedFundPA_1, dtype: float64

Mean Absolute Error (MAE): 2.922377073759073
Mean Squared Error (MSE): 27.35613950902649
Root Mean Squared Error (RMSE): 5.23030969532651
R-squared (R2): 0.6231502329274431


In [21]:
conn.close()